In [4]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/jleland/coding/projects/flopter')
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.utils as spu

In [36]:
# T_e = 5
T_e = 10 #eV
n_0 = 1e18
B = 0.8

In [6]:
coul_log = 12

In [33]:
lambda_D = lpu.debye_length(T_e, n_0)
v_therm = lpu.thermal_velocity(T_e)
v_therm_e = lpu.thermal_velocity(T_e, mass=1/1836)
rho_L = lpu.ion_larmor_radius(T_e, B)
omega_i = lpu.ion_gyrofrequency(B)
omega_e = lpu.electron_gyrofrequency(B)

In [34]:
print(f'lambda_D = {lambda_D}')
print(f'v_i,th = {v_therm}')
print(f'v_e,th = {v_therm_e}')
print(f'rho_L = {rho_L}')

lambda_D = 1.6622799720325184e-05
v_i,th = 21884.73578537274
v_e,th = 937729.6457328595
rho_L = 0.0002855871810057382


In [6]:
rho_L / lambda_D 

17.180450093286176

In [11]:
def coulomb_logarithm_chen(temp, dens):
    return np.log(12 * np.pi * dens * (lpu.debye_length(temp, dens)**3))

def coulomb_logarithm_nrl_ei(temp, dens, z=1):
    n_e = dens * 1e-6
    return 23 - np.log(np.sqrt(n_e) * z * np.float_power(temp, -3/2))

def coulomb_logarithm_nrl_ee(temp, dens):
    n_e = dens * 1e-6
    return 23.5 - np.log(np.sqrt(n_e) * np.float_power(temp, -5/4)) - np.sqrt(1e-5 + ((np.log(temp) - 2)**2)/16)

def coulomb_logarithm_nrl_ii(temp, dens, z_1=1, z_2=1, mu_1=1, mu_2=1, n_2=None, T_2=None):
    n_1 = dens * 1e-6
    T_1 = temp
    if n_2 is None:
        n_2 = n_1
    if T_2 is None:
        T_2 = T_1
    return 23 - np.log(((z_1 * z_2 * (mu_1 + mu_2)) / (mu_1*T_2 + mu_2*T_1)) 
                       * np.sqrt((n_1 * z_1**2 / T_1) + (n_2 * z_2**2 / T_2)))

In [12]:
print(
    f'chen: {coulomb_logarithm_chen(T_e, n_0)} \n'
    f'nrl_ee: {coulomb_logarithm_nrl_ee(T_e, n_0)} \n'
    f'nrl_ei: {coulomb_logarithm_nrl_ei(T_e, n_0)} \n'
    f'nrl_ii: {coulomb_logarithm_nrl_ii(T_e, n_0)} \n'
)

chen: 13.101682996878063 
nrl_ee: 12.487008466771838 
nrl_ei: 12.638367081526795 
nrl_ii: 12.291793491246821 



In [39]:
def collision_freq(temp, dens, mu=1, coulomb_log=10):
    v_therm = lpu.thermal_velocity(temp, mass=mu)
    m = c.PROTON_MASS * mu
    if coulomb_log is None:
        coulomb_log = coulomb_logarithm_chen(temp, dens)
    return coulomb_log * ((np.pi * c.ELEM_CHARGE**4 * dens) 
                          / ((4 * np.pi * c.EPSILON_0)**2 * (m**2) * np.float_power(v_therm, 3)))

def collision_freq_AP(temp, dens):
    return coulomb_logarithm_chen(temp, dens) * ((np.pi * c.ELEM_CHARGE**4 * dens) 
                                                 / ((4 * np.pi * c.EPSILON_0)**2 * np.float_power(c.BOLTZMANN * temp, 3/2)))

In [56]:
def collision_time(temp, dens, coulomb_log=13, mode=None):
    if mode is None:
        mode = 'e'
    if coulomb_log is None:
        coulomb_log = coulomb_logarithm_nrl_ee(temp, dens)
    T_keV = temp / 1000
    if mode == 'e':
        return 1.09e16 * (np.float_power(T_keV, 3/2) / (dens * coulomb_log))
    elif mode == 'i':
        return 6.60e17 * (np.float_power(T_keV, 3/2) / (dens * 1.1 * coulomb_log))

def mean_free_path(temp, dens, coulomb_log=None):
    if coulomb_log is None:
        coulomb_log = coulomb_logarithm_nrl_ee(temp, dens)
    T_keV = temp / 1000
    return 1.44e23 * (T_keV**2 / (dens * coulomb_log))

In [47]:
v_coll_e = collision_freq(T_e, n_0, mu=1/1836, coulomb_log=None)
v_coll_i = collision_freq(T_e, n_0, mu=1, coulomb_log=None)
print(f'{v_coll_e}, {v_coll_i}')

1131809.136026031, 26414.163201636664


In [49]:
L_mfp_e = (v_therm_e / v_coll_e) # / lambda_D
L_mfp_i = (v_therm / v_coll_i)
print(f'{L_mfp_e}, {L_mfp_i}')

0.8285227746308739, 0.8285227746308741


In [42]:
L = 550

In [21]:
L_traversed = np.sqrt(L**2 + (2*np.pi*L)**2)
L_traversed

3499.2458223620306

In [22]:
omega_e

140705601817.9127

In [53]:
tau_e = collision_time(T_e, n_0, coulomb_log=None, mode='e')
tau_i = collision_time(T_e, n_0, coulomb_log=None, mode='i')
print(f'{tau_e}, {tau_i}')

8.729072322650459e-07, 4.804993939073647e-05


In [59]:
print(v_therm_e * tau_e)
print(v_therm * tau_i)
print(mean_free_path(T_e, n_0))

0.8185509896695524
1.0515602280694416
1.1531985453776754


## Try and recreate that plot in Ales' thesis

In [54]:
def calc_L(temp, dens, AP_fl=False, mu=1):
    if AP_fl:
        v_coll = collision_freq_AP(temp, dens)
    else:
        v_coll = collision_freq(temp, dens, mu=mu)
        
    lambda_D = lpu.debye_length(temp, dens)
    v_th = lpu.thermal_velocity(temp, mass=mu)
    L_mfp = (v_therm / v_coll)
    return L_mfp * np.sqrt(1 + (2 * np.pi)) / lambda_D

In [22]:
n_0_arr = np.logspace(17, 20, 61)
n_0_arr

array([1.00000000e+17, 1.12201845e+17, 1.25892541e+17, 1.41253754e+17,
       1.58489319e+17, 1.77827941e+17, 1.99526231e+17, 2.23872114e+17,
       2.51188643e+17, 2.81838293e+17, 3.16227766e+17, 3.54813389e+17,
       3.98107171e+17, 4.46683592e+17, 5.01187234e+17, 5.62341325e+17,
       6.30957344e+17, 7.07945784e+17, 7.94328235e+17, 8.91250938e+17,
       1.00000000e+18, 1.12201845e+18, 1.25892541e+18, 1.41253754e+18,
       1.58489319e+18, 1.77827941e+18, 1.99526231e+18, 2.23872114e+18,
       2.51188643e+18, 2.81838293e+18, 3.16227766e+18, 3.54813389e+18,
       3.98107171e+18, 4.46683592e+18, 5.01187234e+18, 5.62341325e+18,
       6.30957344e+18, 7.07945784e+18, 7.94328235e+18, 8.91250938e+18,
       1.00000000e+19, 1.12201845e+19, 1.25892541e+19, 1.41253754e+19,
       1.58489319e+19, 1.77827941e+19, 1.99526231e+19, 2.23872114e+19,
       2.51188643e+19, 2.81838293e+19, 3.16227766e+19, 3.54813389e+19,
       3.98107171e+19, 4.46683592e+19, 5.01187234e+19, 5.62341325e+19,
      

In [62]:
fig, axes = plt.subplots(1, 2)

for i, ax in enumerate(axes):
    for temp in [1, 2, 5, 10, 20 ,50]:
        ax.plot(n_0_arr, calc_L(temp, n_0_arr, AP_fl=i==1, mu=200/1836), label=temp)

    ax.set_xscale('log')
#     ax.set_yscale('log')
    
    ax.set_xlim(1e17, 1e20)
    ax.legend()

axes[0].set_ylim(0, 100000)

(0, 100000)

In [49]:
fig, ax = plt.subplots()
for temp in [1, 2, 5, 10, 20 ,50]:
    temp_K = temp * c.ELEM_CHARGE / c.BOLTZMANN
    ax.plot(n_0_arr, calc_L(temp_K, n_0_arr, AP_fl=True), label=temp)

    ax.set_xscale('log')
#     ax.set_yscale('log')
    
    ax.set_xlim(1e17, 1e20)
    ax.legend()

In [47]:
5 *c.ELEM_CHARGE / c.BOLTZMANN

58022.61027303315

In [51]:
5e12/600

8333333333.333333

In [52]:
2.74e5 / 600

456.6666666666667